In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Load the CLIP model from TensorFlow Hub
clip_model = hub.load("https://tfhub.dev/google/imagenet/clip_rn50/feature_vector/1")

# Define class labels for breast cancer classification
labels = ["benign", "malignant"]

# Function to preprocess images
def preprocess_image(image_path):
    image = Image.open(image_path).resize((224, 224))
    image = np.array(image) / 255.0
    return tf.expand_dims(image, axis=0)

# Load and preprocess an example breast cancer image
image_path = "breast_cancer_sample.jpg"
image_tensor = preprocess_image(image_path)

# Encode the image using CLIP
image_features = clip_model(image_tensor)

# Encode text descriptions
text_inputs = tf.constant([f"A histopathology image of {label}" for label in labels])
text_features = clip_model.signatures['default'](text_inputs)['default']

# Normalize feature vectors
image_features = tf.nn.l2_normalize(image_features, axis=-1)
text_features = tf.nn.l2_normalize(text_features, axis=-1)

# Compute similarity scores (cosine similarity)
similarity = tf.linalg.matmul(image_features, text_features, transpose_b=True)

# Find the most likely class
predicted_label = labels[np.argmax(similarity.numpy())]

# Print classification result
print(f"Predicted Class: {predicted_label}")

# Display similarity scores
for label, score in zip(labels, similarity.numpy()[0]):
    print(f"{label}: {score:.4f}")
